<a href="https://colab.research.google.com/github/JozefV99/Master-Thesis/blob/main/synthetic_dataset_generation/IMAGE_GENERATION_(BLENDER_CODE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMAGE GENERATION CODE:**

The code below performs image generation for the component "D1098590". This example represents generation for the HIGH COMPLEXITY DATASET but can also be used for other complexities by adjusting materials, backgrounds, and lighting (which is specific to each component). LOW COMPLEXITY images include white background and only one material. MEDIUM COMPLEXITY images include white background and 3 different materials per each component. HIGH COMPLEXITY imgaes include 9 different backgrounds and 3 different materials for each component. For each component the 3D model needs to be manually importend and then the "target_object_name" has to be modified accordingly. Furthermore, lighting parameters ("sun_light.energy", "light_object.data.energy") as well as ("camera.data.lens") need to also be set separatelly for each component. "camera.data.lens" modifies the distance of the object from camera. This is done in order to generate the most accurate pictures.

This python code is meant to be used inside blender and not here. This document was made for explanatory purposes.

In [ ]:
import bpy
import math
import random
import gc

# Define the target object name for easy replacement
target_object_name = "D1098590"

# Set the number of images to generate
num_images = 10000

# Define paths to your background images
background_images = [
    r"C:\Master's Thesis\Data\Composit background\composite_image13.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image13_2.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image13_3.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image14.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image14_2.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image14_3.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image15.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image15_2.png",
    r"C:\Master's Thesis\Data\Composit background\composite_image15_3.png",
    # Add as many paths as you have background images
]

# Automatically generate a list of material names available in Blender
#material_names = [material.name for material in bpy.data.materials]
material_names = [
    "Metal.003",
    "Frozen white metal",
    "Procedural Scratched Metal"
    # Add as many material names as needed
]

# Define path for your HDRI (for lighting only) - this one was chosen for realictic look, there are more to choose from
hdri_path = r"C:\Program Files\Blender Foundation\Blender 4.0\4.0\datafiles\studiolights\world\forest.exr"

# Initialize global variables for object location, dimensions, and camera distance
object_location = (0, 0, 0)
object_dimensions = (0, 0, 0)
bpy.data.objects[target_object_name].rotation_euler = (0, 0, 0)
camera_distance = 0  # This will be set based on the target object

def set_background_image(image_path):
    bpy.context.scene.world.use_nodes = True
    nodes = bpy.context.scene.world.node_tree.nodes
    links = bpy.context.scene.world.node_tree.links

    nodes.clear()
    bg_node = nodes.new(type='ShaderNodeBackground')
    env_texture_node = nodes.new(type='ShaderNodeTexEnvironment')
    env_texture_node.image = bpy.data.images.load(image_path)
    links.new(env_texture_node.outputs['Color'], bg_node.inputs['Color'])
    world_output_node = nodes.new(type='ShaderNodeOutputWorld')
    links.new(bg_node.outputs['Background'], world_output_node.inputs['Surface'])

def setup_hdri_lighting(hdri_path):
    world = bpy.context.scene.world
    world.use_nodes = True
    nodes = world.node_tree.nodes
    links = world.node_tree.links

    # Clear any existing nodes
    nodes.clear()

    # Create nodes for HDRI lighting
    env_texture = nodes.new('ShaderNodeTexEnvironment')
    env_texture.image = bpy.data.images.load(hdri_path)

    # Create a Background node and connect the HDRI for lighting only
    bg = nodes.new('ShaderNodeBackground')
    bg.inputs['Strength'].default_value = 1.0  # Adjust this based on lighting needs
    links.new(env_texture.outputs['Color'], bg.inputs['Color'])

    # Use Light Path to make HDRI visible to lighting calculations only
    light_path = nodes.new('ShaderNodeLightPath')
    is_camera_ray = nodes.new('ShaderNodeMath')
    is_camera_ray.operation = 'GREATER_THAN'
    is_camera_ray.inputs[1].default_value = 0.5  # This will always output 0, making HDRI invisible to camera
    links.new(light_path.outputs['Is Camera Ray'], is_camera_ray.inputs[0])

    # Mix Shader to combine based on camera ray
    mix_shader = nodes.new('ShaderNodeMixShader')
    transparent_bsdf = nodes.new('ShaderNodeBsdfTransparent')
    links.new(is_camera_ray.outputs['Value'], mix_shader.inputs['Fac'])
    links.new(bg.outputs['Background'], mix_shader.inputs[1])
    links.new(transparent_bsdf.outputs['BSDF'], mix_shader.inputs[2])

    # Connect to the World Output
    world_output = nodes.new('ShaderNodeOutputWorld')
    links.new(mix_shader.outputs['Shader'], world_output.inputs['Surface'])

# Call the setup functions
setup_hdri_lighting(hdri_path)

def apply_random_material_to_specific_slot(target_object_name, target_material_name):
    # Select a random material (based on weights) from the available materials
    weights = [60, 20, 20]
    random_material_name = random.choices(material_names, weights=weights, k=1)[0]
    random_material = bpy.data.materials.get(random_material_name)

    # Ensure the random material exists
    if not random_material:
        print(f"Random material '{random_material_name}' not found.")
        return

    # Get the target object
    obj = bpy.data.objects.get(target_object_name)
    if not obj or obj.type != 'MESH':
        print(f"Object '{target_object_name}' not found or is not a mesh.")
        return

    # Find the slot of the target material and replace it
    for i, slot in enumerate(obj.material_slots):
        if slot.material and slot.material.name == target_material_name:
            obj.material_slots[i].material = random_material
            print(f"Replaced material '{target_material_name}' with '{random_material_name}' on '{target_object_name}'.")
            break

def setup_camera_and_initial_lighting(target_object_name):
    global object_location, object_dimensions, camera_distance

    target_object = bpy.data.objects.get(target_object_name)
    if not target_object:
        print(f"Object '{target_object_name}' not found. Please check the object name.")
        return

    object_location = target_object.location
    object_dimensions = target_object.dimensions
    camera_distance = max(object_dimensions) * 5  # Adjust for visibility

    # Delete existing lights except the sun light
    bpy.ops.object.select_by_type(type='LIGHT')
    for obj in bpy.context.selected_objects:
        if obj.data.type != 'SUN':
            bpy.data.objects.remove(obj, do_unlink=True)

    # Setup or adjust the constant sun light
    sun_light = bpy.data.lights.get("ConstantSun") or bpy.data.lights.new(name="ConstantSun", type='SUN')
    sun_light_object = bpy.data.objects.get("ConstantSun")
    if not sun_light_object:
        sun_light_object = bpy.data.objects.new(name="ConstantSun", object_data=sun_light)
        bpy.context.collection.objects.link(sun_light_object)
    sun_light.energy = random.uniform(4, 6)
    sun_light_object.parent = bpy.data.objects.get('Camera')
    sun_light_object.location = (0, 0, 0)

    # Set camera to face the target object automatically
    camera = bpy.data.objects.get('Camera')
    if camera:
        camera.data.lens = 50
        track_to_constraint = camera.constraints.new(type='TRACK_TO')
        track_to_constraint.target = target_object
        track_to_constraint.track_axis = 'TRACK_NEGATIVE_Z'
        track_to_constraint.up_axis = 'UP_Y'
        camera.location = object_location + (camera.location - object_location).normalized() * camera_distance

    # Initialize additional lights with random properties
    for i in range(3):  # Three additional lights
        light_data = bpy.data.lights.new(name=f"RandomLight_{i}", type='POINT')
        light_object = bpy.data.objects.new(name=f"RandomLight_{i}", object_data=light_data)
        bpy.context.collection.objects.link(light_object)

    # Apply preferences adjustments
    bpy.context.preferences.edit.undo_steps = 10
    bpy.context.scene.render.threads_mode = 'FIXED'
    bpy.context.scene.render.threads = 4

def randomize_lights_around_object():
    for i in range(3):  # For each of the three additional lights
        light_object = bpy.data.objects.get(f"RandomLight_{i}")
        if light_object:
            # Random strength and color
            light_object.data.energy = random.uniform(0, 130)  # Adjust the range as needed
            light_object.data.color = (random.random(), random.random(), random.random())

            # Random position around the object within a specified range
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(min(object_dimensions), max(object_dimensions) * 1.5)
            height = random.uniform(-max(object_dimensions), max(object_dimensions))
            light_object.location.x = object_location.x + distance * math.cos(angle)
            light_object.location.y = object_location.y + distance * math.sin(angle)
            light_object.location.z = object_location.z + height

def center_origin_to_geometry(obj):
    # Store current selection
    selected_objects = bpy.context.selected_objects
    active_object = bpy.context.active_object

    # Select and make the target object active
    bpy.ops.object.select_all(action='DESELECT')
    obj.select_set(True)
    bpy.context.view_layer.objects.active = obj

    # Set origin to center of geometry
    bpy.ops.object.origin_set(type='ORIGIN_GEOMETRY', center='BOUNDS')

    # Restore previous selection
    bpy.ops.object.select_all(action='DESELECT')
    for object in selected_objects:
        object.select_set(True)
    bpy.context.view_layer.objects.active = active_object

def purge_unused_data():
    # Purge unused mesh data
    for mesh in bpy.data.meshes:
        if mesh.users == 0:
            bpy.data.meshes.remove(mesh)

    # Purge unused materials
    for material in bpy.data.materials:
        if material.users == 0:
            bpy.data.materials.remove(material)

    # Purge unused textures
    for texture in bpy.data.textures:
        if texture.users == 0:
            bpy.data.textures.remove(texture)

    # Purge unused images
    for image in bpy.data.images:
        if image.users == 0:
            bpy.data.images.remove(image)

    # Optionally, purge orphaned data blocks as well
    bpy.ops.outliner.orphans_purge()

# Initialize the scene setup
setup_camera_and_initial_lighting(target_object_name)

# Main rendering loop with constrained randomness
for i in range(num_images):
    camera = bpy.data.objects['Camera']

    # Set background
    background_image = random.choice(background_images)
    set_background_image(background_image)

    # Apply a constrained range of focal lengths
    camera.data.lens = random.uniform(40, 90)  # Narrower range to reduce extreme zoom effects

    # Calculate a safer angle for camera rotation around the object
    # and apply moderate rotations to the object to ensure it remains mostly visible
    target_object = bpy.data.objects.get(target_object_name)
    if target_object:
        # Center the origin to the object's geometry for accurate rotation
        center_origin_to_geometry(target_object)

        # Apply moderate rotations to ensure the object remains visible
        target_object.rotation_euler = (
            random.uniform(-math.pi / 4, math.pi / 4),  # Limit rotation to [-45°, 45°] range
            random.uniform(-math.pi / 4, math.pi / 4),
            random.uniform(-math.pi / 4, math.pi / 4)
        )

    # Dynamically get the name of the material in the first slot at the beginning of each loop
    target_object = bpy.data.objects.get(target_object_name)
    if target_object and target_object.material_slots:
        first_slot_material_name = target_object.material_slots[0].material.name
    else:
        first_slot_material_name = None  # Fallback if no material is found

    if first_slot_material_name:
        # Randomize and apply a material to the specified slot of the object for each image
        apply_random_material_to_specific_slot(target_object_name, first_slot_material_name)
    else:
        print("No material found in the first slot.")

    # Randomize light positions and properties
    randomize_lights_around_object()

    # Randomly rotate the object
    if target_object:
        target_object.rotation_euler = (
            random.uniform(0, 2 * math.pi),
            random.uniform(0, 2 * math.pi),
            random.uniform(0, 2 * math.pi)
        )


    # Apply periodic data purging
    if i % 100 == 0:
        purge_unused_data()

    # Call garbage collection -> helps to evade crashing, especially on lower-end machines without a proper graphical card (my case)
    gc.collect()

    # Update scene and render
    bpy.context.view_layer.update()

    output_directory = "D:\\HIGH COMPLEXITY\\D1098590\\"  # this needs to be adjusted by the user
    bpy.context.scene.render.filepath = f"{output_directory}D1098590_{i}"

    bpy.ops.render.render(write_still=True)

    # Call garbage collection
    gc.collect()